# Example run: MAGICC

Here we demonstrate how to run with MAGICC. We run with the IPs from WG3 and demonstrate that the results in the database can be reproduced. However, other input data could be used for custom scenario runs.

For more information, see the docs associated with the MAGICC model.

## Setup logging

Pyam does its own logging stuff, so we have to configure logging before that import.

In [1]:
import logging

# Increase the level to reduce the number of log messages
LOGGING_LEVEL = logging.INFO

LOGGER = logging.getLogger("pipeline")
LOGGER.setLevel(LOGGING_LEVEL)
# have to set root logger too to get messages to come through
logging.getLogger().setLevel(LOGGING_LEVEL)

logFormatter = logging.Formatter(
    "%(asctime)s %(name)s %(threadName)s - %(levelname)s:  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
stdoutHandler = logging.StreamHandler()
stdoutHandler.setFormatter(logFormatter)

logging.getLogger().addHandler(stdoutHandler)

## Other imports

In [2]:
import os.path

import dotenv
import matplotlib.pyplot as plt
import pandas as pd
import pandas.testing as pdt
import pooch
import pyam
import tempfile

from climate_assessment.cli import run_workflow

2024-12-06 16:05:49 pint.util MainThread - WARNING:  Redefining 'kt' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2024-12-06 16:05:49 pint.util MainThread - WARNING:  Redefining 'EUR_2005' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2024-12-06 16:05:50 pint.util MainThread - WARNING:  Redefining 'EUR' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
c:\Users\tonnru\lab\envs\scm_conda\Lib\site-packages\scmdata\database\_database.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman
2024-12-06 16:05:51 pint.util MainThread - WARNING:  Redefining 'C' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2024-12-06 16:05:51 pint.util MainThread - WARNING:  Redefining 'N' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2024-12-06 16:05:51 pint.util MainThread - WARNING:  Redefining 'NOX' (

In [3]:
dotenv.load_dotenv()

True

## Configuration of input data

### Download model and configuration

The MAGICC model and configuration is available at [magicc.org](https://magicc.org/), where you can also download the parameter set used in AR6. Please read the license and expectations carefully, we rely on users to act in a way which brings both new scientific outcomes but also acknowledges the work put into the MAGICC AR6 setup.

After you have downloaded the tar files, please extract it (we typically extract MAGICC and the probabilistic distribution into `magicc-files` in the root directory). 

The set of commands on a unix-like system is given below (PRs to add the Windows equivalent are welcome).

In [4]:
# The variables can be taken from an environment (.env) file or simply set here
# Just replace the links with the ones you receive from www.magicc.org
# os.environ["MAGICC_LINK_FROM_MAGICC_DOT_ORG"] = "www.magicc.org/api/download?type=MAGICC7&token=tok"
# os.environ["MAGICC_PROB_DISTRIBUTION_LINK_FROM_MAGICC_DOT_ORG"] = "www.magicc.org/api/download?type=AR6_PROBABILISTIC&token=tok"
# os.environ["MAGICC_ROOT_FILES_DIR"] = os.path.join("..", "magicc-files")
os.environ["MAGICC_ROOT_FILES_DIR"] = r"C:\Users\tonnru\lab\MAGICC7"
print(f'MAGICC_ROOT_FILES_DIR = {os.environ["MAGICC_ROOT_FILES_DIR"]}')

MAGICC_ROOT_FILES_DIR = C:\Users\tonnru\lab\MAGICC7


The cell below downloads and sets up MAGICC on a linux system. It can be skipped once it has been run once (and will need to be modified for a windows system).

In [5]:
# # MAGICC binary
# !mkdir -p "${MAGICC_ROOT_FILES_DIR}"/magicc-v7.5.3
# !wget -O "${MAGICC_ROOT_FILES_DIR}/magicc-v7.5.3.tar.gz" "${MAGICC_LINK_FROM_MAGICC_DOT_ORG}"
# !tar -xf "${MAGICC_ROOT_FILES_DIR}/magicc-v7.5.3.tar.gz" -C "${MAGICC_ROOT_FILES_DIR}/magicc-v7.5.3"

# # Probabilistic distribution
# !mkdir -p "${MAGICC_ROOT_FILES_DIR}/magicc-ar6-0fd0f62-f023edb-drawnset"
# !wget -O "${MAGICC_ROOT_FILES_DIR}/magicc-ar6-0fd0f62-f023edb-drawnset.tar.gz" "${MAGICC_PROB_DISTRIBUTION_LINK_FROM_MAGICC_DOT_ORG}"
# !tar -xf "${MAGICC_ROOT_FILES_DIR}/magicc-ar6-0fd0f62-f023edb-drawnset.tar.gz" -C "${MAGICC_ROOT_FILES_DIR}/magicc-ar6-0fd0f62-f023edb-drawnset"

### Set model input paths

Here we tell the pipeline where to look for MAGICC and where to create parallel workers on disk.

In [6]:
# Where is the binary
os.environ["MAGICC_EXECUTABLE_7"] = os.path.join(
    os.environ["MAGICC_ROOT_FILES_DIR"], "magicc-v7.5.3", "bin", 
    "magicc.exe" if os.name == "nt" else "magicc7"
)

# How many MAGICC workers can run in parallel?
os.environ["MAGICC_WORKER_NUMBER"] = "4"

# Where should the MAGICC workers be located on the filesystem (you need about
# 500Mb space per worker at the moment, they're removed after use)
os.environ["MAGICC_WORKER_ROOT_DIR"] = tempfile.gettempdir()


print(f'MAGICC_EXECUTABLE_7 = {os.environ["MAGICC_EXECUTABLE_7"]}')
print(f'MAGICC_WORKER_ROOT_DIR = {os.environ["MAGICC_WORKER_ROOT_DIR"]}')

MAGICC_EXECUTABLE_7 = C:\Users\tonnru\lab\MAGICC7\magicc-v7.5.3\bin\magicc.exe
MAGICC_WORKER_ROOT_DIR = C:\Users\tonnru\AppData\Local\Temp


### Set general input arguments and options to the climate assessment workflow

The values we've set below will let you run MAGICC and reproduce the AR6 WG3 results.

In [7]:
model = "magicc"
model_version = "v7.5.3"
probabilistic_file = os.path.join(
    os.environ["MAGICC_ROOT_FILES_DIR"],
    "magicc-ar6-0fd0f62-f023edb-drawnset",
    "0fd0f62-derived-metrics-id-f023edb-drawnset.json",
)
probabilistic_file = r"C:\Users\tonnru\lab\MAGICC7\magicc-ar6-0fd0f62-f023edb-drawnset\0fd0f62-derived-metrics-id-f023edb-drawnset.json"
print(os.path.exists(probabilistic_file))

# Use fewer (e.g. 10) if you just want to do a test run but note that this breaks
# the stats of the probabilistic ensemble
num_cfgs = 600
# Set to True if you're not using the full MAGICC ensemble
test_run = False
# How many scenarios do you want to run in one go?
scenario_batch_size = 20

# Where should the output be saved?
outdir = os.path.join("..", "data", "output-magicc-example-notebook")
outdir = os.path.realpath(outdir)
print(os.path.exists(outdir))
print(outdir)

True
True
C:\Users\tonnru\lab\climate-assessment\data\output-magicc-example-notebook


### Choose input emissions pathway file

By default, we use 2 of the AR6 IPs. You could use a different file (formatted the same way) if you wish.

In [8]:
# EMISSIONS_DATA_DIR = os.path.join("..", "tests", "test-data")
# EMISSIONS_INPUT_FILE = "ar6_IPs_emissions.csv"

# input_emissions_file = os.path.join(EMISSIONS_DATA_DIR, EMISSIONS_INPUT_FILE)
input_emissions_file = (
    r"C:\Users\tonnru\lab\climate-assessment\tests\test-data\ar6_IPs_emissions.csv"
)

### Choose infiller database file

We run using the infiller database that was used in CMIP6. As a result of the licensing of the scenario data, this file has to be downloaded by hand (see documentation under "Installation", section "Infiller database"). Make sure that the variable `infilling_database_file` points to where you saved this file.

In [9]:
# infilling_database_file = os.path.join(
#     "..",
#     "data",
#     "1652361598937-ar6_emissions_vetted_infillerdatabase_10.5281-zenodo.6390768.csv",
# )
infilling_database_file = r"C:\Users\tonnru\lab\climate-assessment\tests\test-data\cmip6-ssps-workflow-emissions_infillerdatabase_until2100.csv"

## Run the climate assessment workflow

*N.B. the log with information and some warnings will be quite long - but that is nothing to worry about!*

In [10]:
run_workflow(
    input_emissions_file,
    outdir,
    model=model,
    model_version=model_version,
    probabilistic_file=probabilistic_file,
    num_cfgs=num_cfgs,
    infilling_database=infilling_database_file,
    scenario_batch_size=scenario_batch_size,
)

2024-12-06 16:05:55 climate_assessment.cli MainThread - INFO:  Outputs will be saved in: C:\Users\tonnru\lab\climate-assessment\data\output-magicc-example-notebook
2024-12-06 16:05:55 climate_assessment.cli MainThread - INFO:  Outputs will be saved with the ID: ar6_IPs_emissions
2024-12-06 16:05:55 climate_assessment.cli MainThread - INFO:  Loading C:\Users\tonnru\lab\climate-assessment\tests\test-data\ar6_IPs_emissions.csv
[INFO] 16:05:55 - pyam.core: Reading file C:\Users\tonnru\lab\climate-assessment\tests\test-data\ar6_IPs_emissions.csv
2024-12-06 16:05:55 pyam.core MainThread - INFO:  Reading file C:\Users\tonnru\lab\climate-assessment\tests\test-data\ar6_IPs_emissions.csv
c:\Users\tonnru\lab\envs\scm_conda\Lib\site-packages\pyam\utils.py:318: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence

Note that the lead variable ['Emissions|CO2|Energy and Industrial Processes'] goes negative.


Filling required variables: 100%|██████████| 11/11 [00:01<00:00, 10.02it/s]
2024-12-06 16:06:08 climate_assessment.infilling MainThread - INFO:  Infilling using cruncher <class 'silicone.database_crunchers.rms_closest.RMSClosest'>
2024-12-06 16:06:08 climate_assessment.infilling MainThread - INFO:  Infilling ['Emissions|CCl4', 'Emissions|CFC11', 'Emissions|CFC113', 'Emissions|CFC114', 'Emissions|CFC115', 'Emissions|CFC12', 'Emissions|CH2Cl2', 'Emissions|CH3Br', 'Emissions|CH3CCl3', 'Emissions|CH3Cl', 'Emissions|CHCl3', 'Emissions|HCFC141b', 'Emissions|HCFC142b', 'Emissions|HCFC22', 'Emissions|HFC|HFC152a', 'Emissions|HFC|HFC236fa', 'Emissions|HFC|HFC365mfc', 'Emissions|Halon1202', 'Emissions|Halon1211', 'Emissions|Halon1301', 'Emissions|Halon2402', 'Emissions|NF3', 'Emissions|PFC|C3F8', 'Emissions|PFC|C4F10', 'Emissions|PFC|C5F12', 'Emissions|PFC|C7F16', 'Emissions|PFC|C8F18', 'Emissions|PFC|cC4F8', 'Emissions|SO2F2']
Filling required variables:   0%|          | 0/29 [00:00<?, ?it/s]c:

CalledProcessError: Command '['C:\\Users\\tonnru\\lab\\MAGICC7\\magicc-v7.5.3\\bin\\magicc.exe', '--version']' returned non-zero exit status 3221225595.

### Load results

*N.B The filename will likely have changed if you have run your own scenarios.*

In [ ]:
output = pyam.IamDataFrame(os.path.join(outdir, "ar6_IPs_emissions_alloutput.xlsx"))
output

### Compare with database results

These would normally need to be downloaded, but we include a set in the repository for testing. Here we check that we have reproduced the database results. Obviously, this should be skipped if you have run a custom scenario.

In [ ]:
expected_output_file = os.path.join(
    "..", "tests", "test-data", "expected-output-wg3/two_ips_climate_magicc.xlsx"
)

expected_db_output = pyam.IamDataFrame(expected_output_file).timeseries()

# The database does not necessarily include all the outputs we have
test_output = output.timeseries().loc[expected_db_output.index, :]

# Check that we reproduce values
pdt.assert_frame_equal(
    test_output,
    expected_db_output,
    rtol=1e-3,
    atol=1e-3,
)

### Some basic exploration

Look at the scenario categories.

In [ ]:
output.meta["Category"]

Make a plot of median warming.

In [ ]:
ax = output.filter(variable="*|Surface Temperature (GSAT)|*|50.0th Percentile").plot(
    color="scenario"
)
plt.title("Global warming above the 1850-1900 mean")
ax.set_xlim([1995, 2100])

## Conclusion

That's it! You just ran a a full climate assessment workflow going from emissions to temperature (and more) using the functionality from the climate-assessment package, and then visualised the results.

It is also possible to run from the command line, and build more elaborate workflows. For that, please see the extended documentation.